In [76]:
import sys
sys.path.append('../')

import pandas as pd
import cobra
import re

# Load heterologous reactions
df = pd.read_csv('../results/C00024_C06142_metabolites_reactions.csv')

# Load GSMM
model = cobra.io.load_json_model('../data/original/e_coli_core.json')

# Impose anaerobic conditions
model.reactions.get_by_id('EX_o2_e').bounds = (0, 1000)

# Print number of reactions and metabolites in the model
print('Number of reactions:', len(model.reactions))
print('Number of metabolites:', len(model.metabolites))

Number of reactions: 95
Number of metabolites: 72


In [77]:
from warnings import warn
from cobra import Reaction, Metabolite
from six import string_types
import re

import cobra

# This regular expression finds any single letter compartment enclosed in
# square brackets at the beginning of the string. For example [c] : foo --> bar
compartment_finder = re.compile("^\s*(\[[A-Za-z]\])\s*:*")
def set_stoichiometry_from_string(reaction, reaction_str, verbose=True):
    if reaction._model is None:
        warn("no model found")
        model = None
    else:
        model = reaction._model
    original_str = "" + reaction_str # copy
    found_compartments = compartment_finder.findall(reaction_str)
    if len(found_compartments) == 1:
        compartment = found_compartments[0]
        reaction_str = compartment_finder.sub("", reaction_str)
    else:
        compartment = ""

    
    if "<->" in reaction_str:
        reaction.lower_bound = -1000
        reactant_str, product_str = reaction_str.split("<->")
    elif "<==>" in reaction_str:
        reaction.lower_bound = -1000
        reactant_str, product_str = reaction_str.split("<==>")
    elif "-->" in reaction_str:
        reactant_str, product_str = reaction_str.split("-->")
    elif "->" in reaction_str:
        reactant_str, product_str = reaction_str.split("->")
    elif "<--" in reaction_str:
        reaction.upper_bound = 0
        reaction.lower_bound = -1000
        reactant_str, product_str = reaction_str.split("<--")
    elif "<-" in reaction_str:
        reaction.upper_bound = 0
        reaction.lower_bound = -1000
        reactant_str, product_str = reaction_str.split("<-")
    else:
        raise ValueError("no suitable arrow found in '%s'" % reaction_str)

    for substr, factor in ((reactant_str, -1), (product_str, 1)):
        substr = substr.strip()
        if len(substr) == 0:
            continue
        for term in substr.split("+"):
            term = term.strip()
            if term.lower() == "nothing":
                continue
            if " " in term:
                num_str, met_id = term.split()
                num = float(num_str.lstrip("(").rstrip(")")) * factor
            else:
                met_id = term
                num = factor
            met_id += compartment
            try:
                met = model.metabolites.get_by_id(met_id)
            except KeyError:
                if verbose:
                    print("unknown metabolite '%s' created" % met_id)
                    add_met = met_id
                met = cobra.Metabolite(met_id)
            reaction.add_metabolites({met: num})
    return add_met

for row in range(len(df)):
    r = Reaction(df['Reaction'].iloc[row])
    eq = df['Equation'].iloc[row]
    model.add_reactions([r])
    met_id = set_stoichiometry_from_string(r, eq)
    model.metabolites.get_by_id(met_id).compartment = 'c'

model.metabolites.get_by_id('C06142').compartment = 'e'

unknown metabolite 'C00332' created
unknown metabolite 'C01144' created
unknown metabolite 'C00877' created
unknown metabolite 'C00136' created
unknown metabolite 'C01412' created
unknown metabolite 'C06142' created


In [78]:
# Print number of reactions and metabolites in the model
print('Number of reactions:', len(model.reactions))
print('Number of metabolites:', len(model.metabolites))

Number of reactions: 101
Number of metabolites: 78


In [79]:
for r in df['Reaction']:
    print(model.reactions.get_by_id(r))

R1: 2.0 accoa_c --> C00332 + coa_c
R2: C01144 + nad_c <-- C00332 + h_c + nadh_c
R3: C01144 --> C00877 + h2o_c
R4: C00136 + nad_c <-- C00877 + h_c + nadh_c
R5: C01412 + coa_c + nad_c <-- C00136 + h_c + nadh_c
R6: C01412 + h_c + nadh_c --> C06142 + nad_c


In [88]:
from cobra.flux_analysis import flux_variability_analysis

model.solver = 'scipy'
# model.objective = model.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')
model.objective = model.reactions.get_by_id('R6')
flux_variability_analysis(model)